# Indexer

In [3]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
chunk_size = 1500
chunk_overlap = 0.1 * chunk_size
DEVICE = 'cuda'

docs = TextLoader('config/notes.txt', encoding='utf-8').load()
splitter = CharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
chunks = splitter.split_documents(docs)
del docs

vector_database = FAISS.from_documents(
    documents=chunks,
    embedding=HuggingFaceEmbeddings(
        model_name='config/embedding_model/',
        model_kwargs={'device': DEVICE},
    ),
)
vector_database.save_local('config/knowledge-base/')

# Responder

In [1]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

In [2]:
DEVICE = 'cuda'
k = 3
generator_model = 'gemma3:12b'


vector_database = FAISS.load_local(
    folder_path='config/knowledge-base/',
    embeddings=HuggingFaceEmbeddings(
        model_name='config/embedding_model/',
        model_kwargs={'device': DEVICE},
    ),
    allow_dangerous_deserialization=True
)

def retriever(query):
    docs = vector_database.similarity_search(query)
    context = '\n\n'.join([doc.page_content for doc in docs])
    return {
        'query': query,
        'context': context,
    }

with open('config/prompt.txt', 'r', encoding='utf-8') as f:
    prompt_template = f.read()
prompt = PromptTemplate(template=prompt_template)
llm = ChatOllama(model=generator_model, temperature=0.5, streamming=True)
tutor = retriever | prompt | llm

In [3]:
query = '''
انواع توپولوژی های شبکه را توضیح بده.
'''.strip()

In [4]:
for chunk in tutor.stream(query):
    print(chunk.content, end="", flush=True)

C:\Users\omid\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Okay, let's break down the different types of network topologies, based on the notes you provided.

**What is a "Network Topology"?**

Think of a network topology as the *layout* or *arrangement* of computers and devices in a network. It's how they're physically connected. Here are the types mentioned in the notes:

*   **BUS Topology:** Imagine all the computers connected to a single cable. That's a bus topology. It's simple and cheap to set up, but if that main cable breaks, the whole network goes down. Also, there can be problems if two computers try to send data at the same time.
*   **Ring Topology:** In this setup, computers are connected in a circle.
*   **Star Topology:** This is the most common type. All computers connect to a central point, like a switch or hub. If one computer has a problem, it doesn't affect the rest of the network.
*   **Mesh Topology:** Every computer is connected to every other computer. This is very secure and fast, but it's only practical with a smalle